<a href="https://colab.research.google.com/github/williameclee/trade-cumulus-inversion/blob/main/invtci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trade Cumulus Inversion

## Loading of necessary libraries

In [1]:
import numpy as np


## Speficication of experiment parameters

In [7]:
kc = 60
kcp = kc+1
kcm = kc-1
jc = 81
jt = 2*jc-1
jl = 60

R     = 287
cp    = 1004
kappa = R/cp
gr    = 9.81
Ae    = 6.371e+06 
Omega = 7.292e-05
rom   = Ae * Omega

thetat = 320 # theta (kelvin) at model top
thetab = 300 # theta (kelvin) at model bottom
sigma0 = 4.5 # sigma0 (kPa/K)

pb = 100 # pressure (mb) at model bottom
dp = 10 # pressure (mb) change across inversion
theta10 = 302 # mean theta (kelvin) in inversion
dtheta = 5 # dtheta (kelvin) of inversion
thetac = 305 # thetac (kelvin) of inversion

phis = -np.pi/9 # southern extent of inversion (radians)
phin = np.pi/9 # northern extent of inversion (radians)
phic = 0 # latitude (degrees) of inversion center

dp = dp * 100 # pressure (Pa) change across inversion
pb = pb * 100 # pressure (Pa) at model bottom
sigma0 = sigma0 * 1000 # sigma0 (Pa/K)
pt = pb - sigma0*(thetat-thetab) + dp
dt     = (thetat - thetab) / kc
alpha  = (pb-pt) / pb
beta   = thetab / (thetat-thetab)
cs    = alpha*R*(thetat-thetab)
eps   = 4 * rom*rom / cs

dsc = 1.0 / jc
sc = np.zeros(2*jc+1)
scs = np.zeros(2*jc+1)
fac1 = np.zeros(2*jc+1)
f1 = np.zeros(2*jc+1)
phid = np.zeros(2*jc+1)

## Create coordinates

In [8]:
for j in range(-jc, jc+1):
    sc[j+jc] = j * dsc
    if sc[j+jc] > 1.0:
        sc[j+jc] = 1.0
    if sc[j+jc] < -1.0:
        sc[j+jc] = -1.0
    scs[j+jc] = sc[j+jc] * sc[j+jc]
    fac1[j+jc] = eps * dsc * sc[j+jc]
    f1[j+jc] = fac1[j+jc] * (1.0 - scs[j+jc])
    phid[j+jc] = np.rad2deg(np.arcsin(sc[j+jc]))

dz = 1.0 / kc
tdz = 2.0 * dz
fac2 = 2.0 * dsc * dz * dz
z = np.zeros(kc+1)
theta = np.zeros(kc+1)

for k in range(kc+1):
    z[k] = k * dz
    theta[k] = thetab + k * dt

## Initialise variables

In [10]:
theta1 = np.zeros(2*jc+1)
theta2 = np.zeros(2*jc+1)
pres = np.zeros((2*jc+1, kc+1))
gamma = np.zeros((2*jc+1, kc+1))
refp = np.zeros(kc+1)
ftmp = np.zeros(kc)

for j in range(-jc, jc+1):
    if phid[j+jc] > phin or phid[j+jc] < phis:
        theta1[j+jc] = theta10
    elif phid[j+jc] <= phin and phid[j+jc] > phic:
        fphi = (phin - phid[j+jc]) / (phin - phic)
        theta1[j+jc] = theta10 + dtheta * np.exp(fphi)
    elif phid[j+jc] == phic:
        theta1[j+jc] = theta10 + dtheta
    elif phid[j+jc] < phic and phid[j+jc] >= phis:
        fphi = (phid[j+jc] - phis) / (phic - phis)
        theta1[j+jc] = theta10 + dtheta * np.exp(fphi)
    theta2[j+jc] = theta1[j+jc] + thetac

dz = 1.0 / kc
tdz = 2.0 * dz
pres = np.zeros((2*jc+1, kc+1))
gamma = np.zeros((2*jc+1, kc+1))
refp = np.zeros(kc+1)

for k in range(kc+1):
    for j in range(-jc, jc+1):
        if theta[k] >= theta2[j+jc]:
            pres[j+jc, k] = pb - sigma0 * (theta[k] - thetab) + dp
        elif theta[k] < theta2[j+jc] and theta[k] >= theta1[j+jc]:
            fthet = (theta[k] - theta1[j+jc]) / (theta2[j+jc] - theta1[j+jc])
            pres[j+jc, k] = pb - sigma0 * (theta[k] - thetab) + dp * np.exp(fthet)
        elif theta[k] < theta1[j+jc]:
            pres[j+jc, k] = pb - sigma0 * (theta[k] - thetab)
        pres[j+jc, k] /= pb
        gamma[j+jc, k] = pres[j+jc, k]**(kappa - 1.0)

    sum = 0.0
    for j in range(-jc+1, jc-1, 2):
        sum += pres[j+jc, k] * (sc[j+1] - sc[j-1])
    refp[k] = 0.5 * sum

method = 1
x = np.zeros((2*jc+1, kc+1))
xtmp = np.zeros(kc)

for j in range(-jc+1, jc-1, 2):
    x[j, 0] = cp * thetab / cs
    for k in range(kc):
        ftmp[k] = pres[j, k]**kappa / (kappa * alpha)
    # Call intdde function here if available
    x[j, :] = np.interp(x[j, :], np.arange(kc), xtmp)
    x[j, -1] = x[j, 1] - tdz * x[j, 0] / beta
    x[j, kcp] = x[j, kcm] + tdz * (1.0 - alpha)**kappa / (kappa * alpha)

x[0, :] = 0.0  # Assuming special handling for j = 0

x = np.zeros((2*jc+1, kcp+1))

for j in range(-jc+2, jc-2, 2):
    x[j, :] = sc[j]

/var/folders/bm/k1ffy7j930jdkxs326mj3pt80000gn/T/ipykernel_5372/2096905199.py:37: RuntimeWarning: invalid value encountered in double_scalars
  gamma[j+jc, k] = pres[j+jc, k]**(kappa - 1.0)
/var/folders/bm/k1ffy7j930jdkxs326mj3pt80000gn/T/ipykernel_5372/2096905199.py:51: RuntimeWarning: invalid value encountered in double_scalars
  ftmp[k] = pres[j, k]**kappa / (kappa * alpha)


IndexError: index 61 is out of bounds for axis 1 with size 61